## Zomato web scraping

In [1]:
headers = {
    'authority': 'scrapeme.live',
    'dnt': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}  # this is use to tell zomato website from which source or which user is accessing their website because zomato use secure network

In [2]:
import time
import pandas as pd
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from selenium import webdriver

# Selenium setup
driver = webdriver.Chrome(executable_path=r"C:\Users\Pooja\Desktop\workspace\rebel\chromedriver.exe")
driver.get("https://www.zomato.com/bangalore")
time.sleep(2)

scroll_pause_time = 3
screen_height = driver.execute_script("return window.screen.height;")
i = 1

while True:
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
    i += 1
    time.sleep(scroll_pause_time)

    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    if (screen_height) * i > scroll_height:
        break

soup = BeautifulSoup(driver.page_source, "html.parser")
divs1 = soup.findAll('div', class_='sc-1mo3ldo-0')
divs = divs1[4:]

urls = []
rest_name = []
ratings = []
addresses = []
price = []
cuisine = []
data = pd.DataFrame()

for parent in divs:
    try:
        isCom = True
        try:
            if "Inspiration for your first order" in parent.find("h3").text:
                print("no need first")
                isCom = False
        except Exception as e:
            pass

        try:
            if "Best Food in" in parent.find("h1").text:
                print("no need best")
                isCom = False
        except Exception as e:
            pass

        try:
            if "Top brands for" in parent.find("h3").text:
                print("no need top")
                isCom = False
        except Exception as e:
            pass

        try:
            if "End of search results" in parent.find("h3").text:
                print("no need end")
                isCom = False
                break
        except Exception as e:
            pass

        if isCom:
            divas = parent.findAll('div', class_='jumbo-tracker')
            
            

            for par in divas:
                link_tag = par.find("a")
                base = "https://www.zomato.com"

                if 'href' in link_tag.attrs:
                    link = link_tag.get('href')

                url = urljoin(base, link)
                urls.append(url)

                rating_tag = par.div.a.next_sibling.div.div.div.div.div.div.div.text
                price_tag = par.div.a.next_sibling.p.next_sibling.text
                cuisine_tag = par.div.a.next_sibling.p.text

                ratings.append(rating_tag)
                price.append(price_tag)
                cuisine.append(cuisine_tag)

                driver.get(url)
                time.sleep(3)
                soup = BeautifulSoup(driver.page_source, "html.parser")

                rest_namea = [soup.select('h1.sc-7kepeu-0')[-1].text]
                rest_name.extend(rest_namea)

                cuisine_name = [h.text for h in soup.find_all('h4', class_="sc-1s0saks-15 iSmBPS")]
                priceL = [h1.text for h1 in soup.find_all('span', class_="sc-17hyc2s-1 cCiQWA")]
                description = [h2.text for h2 in soup.find_all('p', class_="sc-1s0saks-12 hcROsL")]
                
                if len(rest_namea) == len(cuisine_name) == len(priceL) == len(description):
                    data1 = pd.DataFrame(zip(rest_namea * len(cuisine_name), cuisine_name, priceL, description))
                    data = data.append(data1, ignore_index=True)
                else:
                    print("Skipping data entry due to inconsistent lengths")


                if data.empty:

                    data = pd.DataFrame(zip(rest_namea * len(cuisine_name), cuisine_name, priceL, description))
                else:
                    data1 = pd.DataFrame(zip(rest_namea * len(cuisine_name), cuisine_name, priceL, description))
                    data = data.append(data1, ignore_index=True)

                data.to_csv('food_menu.csv', index=False)

                addresses.append(soup.findAll('a', class_='sc-cmTdod')[0].text)


    except Exception as e:
        print(e)
        pass

out_df2 = pd.DataFrame(
    {'links': urls, 'names': rest_name, 'address': addresses, 'ratings': ratings, 'price for two': price,
     'cuisine': cuisine})

out_df2.to_csv("restaurants_Banglre.csv")
driver.close()


C:\Users\Pooja\AppData\Local\Temp\ipykernel_10276\2385937608.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\Pooja\Desktop\workspace\rebel\chromedriver.exe")


no need first
no need top
no need best
Skipping data entry due to inconsistent lengths
[Errno 13] Permission denied: 'food_menu.csv'
Skipping data entry due to inconsistent lengths
[Errno 13] Permission denied: 'food_menu.csv'


C:\Users\Pooja\AppData\Local\Temp\ipykernel_10276\2385937608.py:115: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(data1, ignore_index=True)


Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out o

Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out o

Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out o

Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out of range
Skipping data entry due to inconsistent lengths
list index out o

ValueError: All arrays must be of the same length